# Ce script à pour vocation d'importer une extraction  Aurion dans la base de données des utilisateurs

In [ ]:
!python -m pip install --upgrade firebase-admin pandas

     ---------------------------------------- 0.0/119.5 kB ? eta -:--:--
     -------------------------------------- 119.5/119.5 kB 6.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
     ------------------ -------------------- 5.0/10.4 MB 155.5 MB/s eta 0:00:01
     --------------------------------- ----- 9.0/10.4 MB 115.7 MB/s eta 0:00:01
     --------------------------------------- 10.4/10.4 MB 81.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
     ------------------- ------------------- 5.6/11.0 MB 118.1 MB/s eta 0:00:01
     ------------------------------------ - 10.6/11.0 MB 108.8 MB/s eta 0:00:01
     --------------------------------------- 11.0/11.0 MB 72.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/265.5 kB ? eta -:--:--
     ---------------------------------------- 265.5/265.5 kB ? eta 0:00:00
     ---------------------------------------- 0.0/110.2 kB ? eta -:--:--
  

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json
import requests

## Connexion à firestore

In [4]:
cred = credentials.Certificate("cred.json")

In [5]:
firebase_admin.initialize_app(cred)

db = firestore.client()

## Accès à la collection utilisateur

In [6]:
users_col = db.collection(u'users')

### Liste users

In [7]:
users_id = []
for snap in users_col.get():
    users_id.append(snap.id)
    
users_id

[]

# Récupération liste des élèves à Aurion

## Connexion à Aurion

In [43]:
aurion_data = {}
with open('aurion.json', 'r') as f:
  aurion_creds = json.load(f)

In [1]:
request_id = 33654306
payload = f"""
<executeFavori>
    <favori>
        <id>{request_id}</id>
    </favori>
    <database>{aurion_creds['database']}</database>
</executeFavori>
"""

data = dict(data=payload, login=aurion_creds['username'], password=aurion_creds['password'])
response = requests.post(aurion_creds['url'], data=data)

print(response.content)

NameError: name 'aurion_creds' is not defined

## Recuperation des informations utilisateur

## Suppression des doublons

In [44]:
users = users.groupby("login.Individu").first().reset_index()
users

,login.Individu,Coordonnée.Coordonnée,Code.Groupe
0,abatouyn,nabila.abatouy@edu.esiee.fr,20_E3FG
1,abboudmg,georges.abboudmansour@edu.esiee.fr,20_E4
2,abdellah,hicham.abdellaoui@edu.esiee.fr,20_E5F_5A_AN1B_IF
3,abdoulaj,jassim.abdoulalime@edu.esiee.fr,20_E4FE
4,abdouna,amir.abdoun@edu.esiee.fr,20_E1
...,...,...,...
2175,zouainea,anas.zouaine@edu.esiee.fr,20_E5FG
2176,zouaouig,gibril.zouaoui@edu.esiee.fr,20_E3
2177,zouf,fangming.zou@edu.esiee.fr,20_E4_PIM
2178,zouhirk,kylian.zouhir@edu.esiee.fr,20_E5F_5A_SH2_1RC


## Simplification des groupes

In [59]:
users["Code.Groupe"] = users["Code.Groupe"].str.split("_").str[:2].str.join("_")
users

,login.Individu,Coordonnée.Coordonnée,Code.Groupe
0,abatouyn,nabila.abatouy@edu.esiee.fr,20_E3FG
1,abboudmg,georges.abboudmansour@edu.esiee.fr,20_E4
2,abdellah,hicham.abdellaoui@edu.esiee.fr,20_E5F
3,abdoulaj,jassim.abdoulalime@edu.esiee.fr,20_E4FE
4,abdouna,amir.abdoun@edu.esiee.fr,20_E1
...,...,...,...
2175,zouainea,anas.zouaine@edu.esiee.fr,20_E5FG
2176,zouaouig,gibril.zouaoui@edu.esiee.fr,20_E3
2177,zouf,fangming.zou@edu.esiee.fr,20_E4
2178,zouhirk,kylian.zouhir@edu.esiee.fr,20_E5F


## Extraction des noms et prenoms

In [64]:
users["surname"] = users["Coordonnée.Coordonnée"].str.split(".").str[0].str.capitalize()
users

,login.Individu,Coordonnée.Coordonnée,Code.Groupe,surname
0,abatouyn,nabila.abatouy@edu.esiee.fr,20_E3FG,Nabila
1,abboudmg,georges.abboudmansour@edu.esiee.fr,20_E4,Georges
2,abdellah,hicham.abdellaoui@edu.esiee.fr,20_E5F,Hicham
3,abdoulaj,jassim.abdoulalime@edu.esiee.fr,20_E4FE,Jassim
4,abdouna,amir.abdoun@edu.esiee.fr,20_E1,Amir
...,...,...,...,...
2175,zouainea,anas.zouaine@edu.esiee.fr,20_E5FG,Anas
2176,zouaouig,gibril.zouaoui@edu.esiee.fr,20_E3,Gibril
2177,zouf,fangming.zou@edu.esiee.fr,20_E4,Fangming
2178,zouhirk,kylian.zouhir@edu.esiee.fr,20_E5F,Kylian


In [66]:
users["name"] = users["Coordonnée.Coordonnée"].str.split(".").str[1].str.split("@").str[0].str.capitalize()
users

,login.Individu,Coordonnée.Coordonnée,Code.Groupe,surname,name
0,abatouyn,nabila.abatouy@edu.esiee.fr,20_E3FG,Nabila,Abatouy
1,abboudmg,georges.abboudmansour@edu.esiee.fr,20_E4,Georges,Abboudmansour
2,abdellah,hicham.abdellaoui@edu.esiee.fr,20_E5F,Hicham,Abdellaoui
3,abdoulaj,jassim.abdoulalime@edu.esiee.fr,20_E4FE,Jassim,Abdoulalime
4,abdouna,amir.abdoun@edu.esiee.fr,20_E1,Amir,Abdoun
...,...,...,...,...,...
2175,zouainea,anas.zouaine@edu.esiee.fr,20_E5FG,Anas,Zouaine
2176,zouaouig,gibril.zouaoui@edu.esiee.fr,20_E3,Gibril,Zouaoui
2177,zouf,fangming.zou@edu.esiee.fr,20_E4,Fangming,Zou
2178,zouhirk,kylian.zouhir@edu.esiee.fr,20_E5F,Kylian,Zouhir


## Ajout des points par défaut

In [67]:
users["point"] = 0
users

,login.Individu,Coordonnée.Coordonnée,Code.Groupe,surname,name,point
0,abatouyn,nabila.abatouy@edu.esiee.fr,20_E3FG,Nabila,Abatouy,0
1,abboudmg,georges.abboudmansour@edu.esiee.fr,20_E4,Georges,Abboudmansour,0
2,abdellah,hicham.abdellaoui@edu.esiee.fr,20_E5F,Hicham,Abdellaoui,0
3,abdoulaj,jassim.abdoulalime@edu.esiee.fr,20_E4FE,Jassim,Abdoulalime,0
4,abdouna,amir.abdoun@edu.esiee.fr,20_E1,Amir,Abdoun,0
...,...,...,...,...,...,...
2175,zouainea,anas.zouaine@edu.esiee.fr,20_E5FG,Anas,Zouaine,0
2176,zouaouig,gibril.zouaoui@edu.esiee.fr,20_E3,Gibril,Zouaoui,0
2177,zouf,fangming.zou@edu.esiee.fr,20_E4,Fangming,Zou,0
2178,zouhirk,kylian.zouhir@edu.esiee.fr,20_E5F,Kylian,Zouhir,0


In [73]:
users = users.rename(columns={
    "login.Individu": "id",
    "Coordonnée.Coordonnée": "email",
    "Code.Groupe": "classe",
})
users

,id,email,classe,surname,name,point
0,abatouyn,nabila.abatouy@edu.esiee.fr,20_E3FG,Nabila,Abatouy,0
1,abboudmg,georges.abboudmansour@edu.esiee.fr,20_E4,Georges,Abboudmansour,0
2,abdellah,hicham.abdellaoui@edu.esiee.fr,20_E5F,Hicham,Abdellaoui,0
3,abdoulaj,jassim.abdoulalime@edu.esiee.fr,20_E4FE,Jassim,Abdoulalime,0
4,abdouna,amir.abdoun@edu.esiee.fr,20_E1,Amir,Abdoun,0
...,...,...,...,...,...,...
2175,zouainea,anas.zouaine@edu.esiee.fr,20_E5FG,Anas,Zouaine,0
2176,zouaouig,gibril.zouaoui@edu.esiee.fr,20_E3,Gibril,Zouaoui,0
2177,zouf,fangming.zou@edu.esiee.fr,20_E4,Fangming,Zou,0
2178,zouhirk,kylian.zouhir@edu.esiee.fr,20_E5F,Kylian,Zouhir,0


In [75]:
users.to_csv("export.csv", index=False)

## Import dans Firestore

In [82]:
nb_user = users.shape[0]
print(nb_user, ' utilisateurs à importer')

for index, user in users.iterrows():
    users_col.document(user["id"]).set({
        u'name': user["name"],
        u'surname': user["surname"],
        u'email': user["email"],
        u'point': user["point"],
        u'classe': user["classe"],
    })
    if index/nb_user == index//nb_user:
        print(index//nb_user, "%")

2180  utilisateurs à importer
0 %
